# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 17 2022 9:32AM,248716,19,CLI00402.1,"CLINUVEL, Inc.",Released
1,Oct 17 2022 9:30AM,248715,10,CLI00403.1,"CLINUVEL, Inc.",Released
2,Oct 17 2022 9:26AM,248711,10,0086076928,ISDIN Corporation,Released
3,Oct 17 2022 9:26AM,248711,10,0086076927,ISDIN Corporation,Released
4,Oct 17 2022 9:26AM,248711,10,0086076932,ISDIN Corporation,Released
5,Oct 17 2022 9:26AM,248711,10,0086076933,ISDIN Corporation,Released
6,Oct 17 2022 9:26AM,248711,10,0086076935,ISDIN Corporation,Released
7,Oct 17 2022 9:26AM,248711,10,0086076936,ISDIN Corporation,Released
8,Oct 17 2022 9:26AM,248711,10,0086076940,ISDIN Corporation,Released
9,Oct 17 2022 9:26AM,248711,10,0086076939,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,248709,Released,1
13,248710,Released,3
14,248711,Released,56
15,248715,Released,1
16,248716,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248709,NaN,NaN,1.0
248710,NaN,NaN,3.0
248711,NaN,NaN,56.0
248715,NaN,NaN,1.0
248716,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248690,1.0,5.0,1.0
248697,0.0,0.0,1.0
248698,0.0,0.0,6.0
248699,0.0,0.0,7.0
248702,0.0,0.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248690,1,5,1
248697,0,0,1
248698,0,0,6
248699,0,0,7
248702,0,0,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248690,1,5,1
1,248697,0,0,1
2,248698,0,0,6
3,248699,0,0,7
4,248702,0,0,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248690,1,5,1
1,248697,,,1
2,248698,,,6
3,248699,,,7
4,248702,,,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 17 2022 9:32AM,248716,19,"CLINUVEL, Inc."
1,Oct 17 2022 9:30AM,248715,10,"CLINUVEL, Inc."
2,Oct 17 2022 9:26AM,248711,10,ISDIN Corporation
58,Oct 17 2022 9:19AM,248710,19,"Graystone, LLC"
61,Oct 17 2022 9:13AM,248707,12,Hush Hush
116,Oct 17 2022 9:12AM,248709,19,"Granules Pharmaceuticals, Inc."
117,Oct 17 2022 9:07AM,248708,10,Bio-PRF
118,Oct 17 2022 9:05AM,248706,10,ISDIN Corporation
130,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC"
131,Oct 17 2022 8:59AM,248704,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 17 2022 9:32AM,248716,19,"CLINUVEL, Inc.",,,1
1,Oct 17 2022 9:30AM,248715,10,"CLINUVEL, Inc.",,,1
2,Oct 17 2022 9:26AM,248711,10,ISDIN Corporation,,,56
3,Oct 17 2022 9:19AM,248710,19,"Graystone, LLC",,,3
4,Oct 17 2022 9:13AM,248707,12,Hush Hush,,,55
5,Oct 17 2022 9:12AM,248709,19,"Granules Pharmaceuticals, Inc.",,,1
6,Oct 17 2022 9:07AM,248708,10,Bio-PRF,,,1
7,Oct 17 2022 9:05AM,248706,10,ISDIN Corporation,,,12
8,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC",,,1
9,Oct 17 2022 8:59AM,248704,10,Emerginnova,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 17 2022 9:32AM,248716,19,"CLINUVEL, Inc.",1,,
1,Oct 17 2022 9:30AM,248715,10,"CLINUVEL, Inc.",1,,
2,Oct 17 2022 9:26AM,248711,10,ISDIN Corporation,56,,
3,Oct 17 2022 9:19AM,248710,19,"Graystone, LLC",3,,
4,Oct 17 2022 9:13AM,248707,12,Hush Hush,55,,
5,Oct 17 2022 9:12AM,248709,19,"Granules Pharmaceuticals, Inc.",1,,
6,Oct 17 2022 9:07AM,248708,10,Bio-PRF,1,,
7,Oct 17 2022 9:05AM,248706,10,ISDIN Corporation,12,,
8,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC",1,,
9,Oct 17 2022 8:59AM,248704,10,Emerginnova,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 17 2022 9:32AM,248716,19,"CLINUVEL, Inc.",1,,
1,Oct 17 2022 9:30AM,248715,10,"CLINUVEL, Inc.",1,,
2,Oct 17 2022 9:26AM,248711,10,ISDIN Corporation,56,,
3,Oct 17 2022 9:19AM,248710,19,"Graystone, LLC",3,,
4,Oct 17 2022 9:13AM,248707,12,Hush Hush,55,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 17 2022 9:32AM,248716,19,"CLINUVEL, Inc.",1,NaN,NaN
1,Oct 17 2022 9:30AM,248715,10,"CLINUVEL, Inc.",1,NaN,NaN
2,Oct 17 2022 9:26AM,248711,10,ISDIN Corporation,56,NaN,NaN
3,Oct 17 2022 9:19AM,248710,19,"Graystone, LLC",3,NaN,NaN
4,Oct 17 2022 9:13AM,248707,12,Hush Hush,55,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 17 2022 9:32AM,248716,19,"CLINUVEL, Inc.",1,0.0,0.0
1,Oct 17 2022 9:30AM,248715,10,"CLINUVEL, Inc.",1,0.0,0.0
2,Oct 17 2022 9:26AM,248711,10,ISDIN Corporation,56,0.0,0.0
3,Oct 17 2022 9:19AM,248710,19,"Graystone, LLC",3,0.0,0.0
4,Oct 17 2022 9:13AM,248707,12,Hush Hush,55,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2487038,92,5.0,1.0
12,248707,55,0.0,0.0
19,746135,5,0.0,0.0
20,248697,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2487038,92,5.0,1.0
1,12,248707,55,0.0,0.0
2,19,746135,5,0.0,0.0
3,20,248697,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,92,5.0,1.0
1,12,55,0.0,0.0
2,19,5,0.0,0.0
3,20,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,92.0
1,12,Released,55.0
2,19,Released,5.0
3,20,Released,1.0
4,10,Executing,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20
Status,,,,
Completed,1.0,0.0,0.0,0.0
Executing,5.0,0.0,0.0,0.0
Released,92.0,55.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20
0,Completed,1.0,0.0,0.0,0.0
1,Executing,5.0,0.0,0.0,0.0
2,Released,92.0,55.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20
0,Completed,1.0,0.0,0.0,0.0
1,Executing,5.0,0.0,0.0,0.0
2,Released,92.0,55.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()